In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()

True

## Langchain Basics

In [2]:
llm=ChatOpenAI()

In [11]:
llm.invoke("What is langsmith?")

AIMessage(content='Langsmith is a surname of English origin. It is derived from the Old English words "lang," meaning long, and "smith," meaning someone who works with metal. This surname was likely given to someone who was a blacksmith or who worked with metal in some way.')

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage

# Prompt template with HumanMessage and AIMessage
prompt = ChatPromptTemplate.from_messages([
    HumanMessage(content="What is langchain?"),
    AIMessage(content="Langchain is a platform for building and deploying language models."),
    HumanMessage(content="{input}")
])

In [19]:
chain = prompt | llm
chain.invoke({"input":"Tell me more about it."})

AIMessage(content='I\'m sorry, but I do not have any additional information on "langchain" as it appears to be a made-up term or a very niche topic with limited publicly available information. If you have more context or details about what you are referring to, I would be happy to try to provide more assistance.')

In [4]:
from langchain_core.output_parsers import StrOutputParser

# String output parser
output_parser = StrOutputParser()

In [21]:
# Chain with prompt, llm and output_parser
chain = prompt | llm | output_parser
chain.invoke({"input":"Tell me more about it."})

'Langchain is a decentralized blockchain-based platform that aims to revolutionize the language industry by providing a secure and transparent ecosystem for language learning, translation, and communication. It leverages blockchain technology to create a decentralized marketplace where users can buy and sell language-related services, such as translation, interpretation, and language tutoring, without the need for intermediaries. The platform also features a token economy that incentivizes users to contribute to the community and rewards them for their participation. Overall, Langchain aims to make language learning and communication more accessible, efficient, and secure for people around the world.'

In [22]:
# Streaming output
for chunk in chain.stream({"input":"Tell me more about it."}):
    print(chunk, end="", flush=True)

I'm sorry, but I don't have any specific information on "langchain" as it seems to be a relatively unknown term or concept. If you have any further context or details about what you are referring to, I would be happy to try to provide more information or assistance.

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()

In [6]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [7]:
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
documents = text_splitter.split_documents(docs)
vector = Chroma.from_documents(documents, embeddings)

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
                                          <context>
                                          {context}
                                          </context>
                                          Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [9]:
from langchain.chains import create_retrieval_chain

# Retrieve k relevant documents
retriever = vector.as_retriever(search_kwargs={"k": 2})
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [93]:
response = retrieval_chain.invoke({"input": "Tell me more about langchain?"})
print(response['answer'])

KeyError: "Input to ChatPromptTemplate is missing variables {'chat_history'}.  Expected: ['chat_history', 'context', 'input'] Received: ['input', 'context']"

In [94]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [95]:
chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes")
]

# retriever_chain.invoke({"chat_history": chat_history, "input": "Tell me step by step."})

In [97]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the following question based only on the provided context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

response = retrieval_chain.invoke({"chat_history":chat_history, "input": "Im Timothy, Tell me step by step."})
print(response['answer'])

1. Utilize LangSmith, the developer platform, for debugging, testing, evaluating, and monitoring chains.
2. Develop your LangChain chains using LangServe, a tool for deploying them as REST APIs.
3. Integrate LangSmith into your development process to test your LLM applications effectively.
4. Follow the comprehensive documentation provided for LangChain to ensure a thorough understanding of its various aspects.
5. By following these steps, you can utilize LangSmith to test your LLM applications efficiently.


In [14]:
from langchain.tools.retriever import create_retriever_tool
retreiver_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information relevant to Langsmith. For any questions about LangSmith, you must use this tool!")

In [15]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [16]:
tools = [retreiver_tool, search]

from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
agent_executor.invoke({"input": "What can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'What can LangSmith help with testing?'}`


LangSmith | 🦜️🛠️ LangSmith

Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLMLangSmith can help with testing by supporting workflows at each stage of the LLM (Language Model Lifecycle). You can learn more about the specific workflows and how LangSmith can assist in the User Guide.

> Finished chain.


{'input': 'What can langsmith help with testing?',
 'output': 'LangSmith can help with testing by supporting workflows at each stage of the LLM (Language Model Lifecycle). You can learn more about the specific workflows and how LangSmith can assist in the User Guide.'}

In [18]:
agent_executor.invoke({"input": "what is the weather like today in singapore ?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Singapore today'}`


[{'url': 'https://www.weatherapi.com/', 'content': "Weather in Singapore is {'location': {'name': 'Singapore', 'region': '', 'country': 'Singapore', 'lat': 1.29, 'lon': 103.86, 'tz_id': 'Asia/Singapore', 'localtime_epoch': 1708789633, 'localtime': '2024-02-24 23:47'}, 'current': {'last_updated_epoch': 1708789500, 'last_updated': '2024-02-24 23:45', 'temp_c': 27.0, 'temp_f': 80.6, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 330, 'wind_dir': 'NNW', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 79, 'cloud': 75, 'feelslike_c': 30.3, 'feelslike_f': 86.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 19.7, 'gust_kph': 31.6}}"}, {'url': 'https://weatherspark.com/h/y/114655/2024/H

{'input': 'what is the weather like today in singapore ?',
 'output': 'The weather in Singapore today is partly cloudy with a temperature of 27.0°C (80.6°F). The wind speed is 3.8 mph and the humidity is 79%. It is currently nighttime in Singapore.'}

## Prompt + LLM

In [139]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser
prompt = ChatPromptTemplate.from_template("""Tell me about {subject}""")
chain = prompt | llm | output_parser
chain.invoke({"subject":"typescript"})

'TypeScript is a programming language developed by Microsoft that is a superset of JavaScript. It adds static typing to the language, making it easier to catch errors and bugs in the code during development. TypeScript also includes features such as interfaces, classes, and modules, which help organize and structure code in a more maintainable way.\n\nOne of the main benefits of using TypeScript is that it allows developers to write cleaner and more scalable code, as the type system helps to enforce consistency and reduce the likelihood of runtime errors. Additionally, TypeScript code can be compiled into plain JavaScript, which means it can be run on any browser or platform that supports JavaScript.\n\nOverall, TypeScript is a popular choice for developers who want to take advantage of the benefits of static typing while still working within the familiar syntax and ecosystem of JavaScript.'

In [137]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]

chain = prompt | llm.bind(function_call={"name":"joke"}, functions=functions) | JsonOutputFunctionsParser()

In [138]:
chain.invoke({"subject":"joke about dog"})

{'setup': 'Why did the dog sit in the shade?',
 'punchline': "Because he didn't want to be a hot dog!"}

In [141]:
chain = prompt | llm.bind(function_call={"name":"joke"}, functions=functions) | JsonKeyOutputFunctionsParser(key_name="punchline")
chain.invoke({"subject":"joke about dog"})

'Because he had perfect pitch!'

In [144]:
from langchain_core.runnables import RunnablePassthrough

chain = {"subject": RunnablePassthrough()} | prompt | llm.bind(function_call={"name":"joke"}, functions=functions) | JsonKeyOutputFunctionsParser(key_name="punchline")

In [145]:
chain.invoke("Singapore")

'He wanted a well-balanced meal!'

## RAG with Qdrant Cloud

In [19]:
import qdrant_client
from qdrant_client.http import models

In [20]:
os.getenv("QDRANT_HOST")

'https://d57352dd-722d-437b-8250-674f0ba883f3.us-east4-0.gcp.cloud.qdrant.io:6333'

In [21]:
client = qdrant_client.QdrantClient(os.getenv("QDRANT_HOST"), api_key=os.getenv("QDRANT_API_KEY"))

In [22]:
vectors_config = models.VectorParams(size=1536, distance=models.Distance.COSINE)
# create collection
client.recreate_collection(collection_name="langsmith_cookbook", vectors_config=vectors_config)

True

In [44]:
from langchain_community.vectorstores import Qdrant
vector = Qdrant(client, "langsmith_cookbook", embeddings=embeddings)
retriever = vector.as_retriever(search_kwargs={"k": 3})

In [45]:
from langchain_community.document_loaders.text import TextLoader
loader = TextLoader("./langchain_docs.txt")
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=20)
docs = text_splitter.split_documents(document)
vector.add_documents(docs)

['03e55596c9b641a7a30184147dd870f4',
 'f1ae83fc93284b928055dc53e22ceef4',
 '59e483e9e6254d538b8007a5866c7fda',
 '1a75cac9a17e4a579684cb6e62250b05',
 '7aa9b47010ed4a1fb4099063bd9bbf12',
 '47e652c6a91742c18003bf0e7ef7659b',
 'd0c905a7440a4aafad146759ca5b0e34',
 '3a9a9fc9126649f58c0344360538a799',
 '2f9ac9a9637e4228862d66ab36e7ebe9',
 'd5d78a760d1943e9bbe39775f6b2c74a',
 'd387a07d0ed74c3e83c5dd2605cae0c4',
 'd10fcf8fa3994ebb97ccad440f9cc302',
 '7aaeb53d78794b1d9f274b3de32f0679',
 '62c8cff3524e4b9799db4e384f7402f3']

In [46]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
                                          <context>
                                          {context}
                                          </context>
                                          Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "What link to find for langchain quickstart?"})

In [47]:
response['answer']

'The link to find for LangChain quickstart is: https://python.langchain.com/docs/get_started/quickstart'

## Memory in Langchain (LCEL)